In [1]:
import pandas as pd
import os
import requests
import json
from dotenv import load_dotenv

Variables List + Header API

In [2]:
load_dotenv()
RIOT_API_KEY = os.getenv("RIOT_API_KEY")
my_game_name = os.getenv("my_game_name")
my_tag_line = os.getenv("my_tag_line")

In [5]:
headers = {
    "X-Riot-Token" : RIOT_API_KEY
}

# Creating a quick data frame just for the purpose of testing the functions in the blocks below

In [4]:
api_url = "https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"

resp = requests.get(api_url, headers=headers)
response = resp.json()
entries = response.get('entries', [])
entries = entries[:4]
test_df = pd.DataFrame([{"summonerId" : entry["summonerId"]} for entry in entries])
test_df.to_csv("test.csv", index = False)

# Function to read csv above and populate it (and the dataframe) with puuids in a new column

In [6]:
def get_puuid(csv_only_summ_ids):
    # Assigning csv file here as a placeholder, can just include it in the argument when using this fucntion
    df_only_summ_ids = pd.read_csv(csv_only_summ_ids, index_col=False)
    # Loop to iterate through each row
    for index, row in df_only_summ_ids.iterrows():
            summoner_id = row["summonerId"]
            # Variable to insert summ_id in API url that fetches account data
            api_url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"
            try:
                resp = requests.get(api_url, headers=headers)  #Insert API link here
                resp.raise_for_status()
                summoner_info = resp.json()
                # Create a column in the data frame called puuids and populate it with the puuid for each summoner id
                df_only_summ_ids.at[index, "puuid"] = summoner_info.get("puuid", "Not Found") # Store puuid in df, output Not Found if KeyError
            # Alternative way to handle request exceptions that handles all request Errors instead of just error 400
            except requests.RequestException as e:
                print(f"Error fetching puuid for {summoner_id}: {e}")
    # Update csv file with the new column
    df_only_summ_ids.to_csv(csv_only_summ_ids, index = False)
# Calling function to add new info to existing csv instead of creating new one (using test csv from above in this case)
get_puuid("test.csv")

# Function to get match ids using puuids from Dataframe

In [ ]:
def get_match_ids(csv_summids_and_puuids, current_patch):
    # Read csv file with summoner ids and puuids 
    df_summoners = pd.read_csv(csv_summids_and_puuids)
    # Create a new DataFrame to store match data
    df_all_matches = pd.DataFrame(columns = ["summonerId", "puuid", "matchId", "matchData"])
    # Variable for number of matches, will be defaulted to max (100) in actual code, small number used for testing
    number_of_matches = 5
    # Iterate through each row in df_summoners
    for _, row in df_summoners.iterrows():
        summoner_id = row["summonerId"]
        puuid = row["puuid"]
        # Fetch match ids for each puuid
        match_history_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={number_of_matches}"

        try:
            resp = requests.get(match_history_api_url, headers=headers)
            resp.raise_for_status()
            match_ids = resp.json()
            # Iterate through each match id and fetch match data
            for match_id in match_ids:
                match_data_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
                response = requests.get(match_data_api_url, headers=headers)
                match_data = response.json()
                # Variable to check for patch/version
                game_patch = (match_data["info"]["gameVersion"].split("."))[0] + "." + (match_data["info"]["gameVersion"].split("."))[1]
                if game_patch == current_patch:
                    # Store match data in JSON file along with summoner id and puuid
                    df_mini = pd.DataFrame({
                        "summonerId" : [summoner_id],
                        "puuid" : [puuid],
                        "matchId" : [match_id],
                        "matchData": [json.dumps(match_data)]
                        })
                    # Append to main DataFrame
                    df_all_matches = pd.concat([df_all_matches, df_mini], ignore_index = True)
                else:
                    continue
        # Alternative way to handle request exceptions that handles all request Errors instead of just error 400
        except requests.RequestException as e:
            print(f"Error fetching matches for {puuid}: {e}")
    df_all_matches = df_all_matches.drop_duplicates("matchId")
    # Save the expanded DataFrame
    df_all_matches.to_csv("all_match_data_test.csv", index = False)
    return df_all_matches

In [ ]:
# Running get_match function, argument 1 = csv file, argument 2 = current patch
get_match_ids("test.csv", "15.3")

,summonerId,puuid,matchId,matchData
0,ykcp8Dwwqm33s1Q9_55qCmuJxx4Hi5cXR9yExXFPVQCYL1c,sdgTQOOEEBvxsKzEYUE4Vll5Pf8wC6So-27l9qDSrV5Gaf...,NA1_5226861599,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
1,ykcp8Dwwqm33s1Q9_55qCmuJxx4Hi5cXR9yExXFPVQCYL1c,sdgTQOOEEBvxsKzEYUE4Vll5Pf8wC6So-27l9qDSrV5Gaf...,NA1_5226826880,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
2,ykcp8Dwwqm33s1Q9_55qCmuJxx4Hi5cXR9yExXFPVQCYL1c,sdgTQOOEEBvxsKzEYUE4Vll5Pf8wC6So-27l9qDSrV5Gaf...,NA1_5226791216,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
3,ykcp8Dwwqm33s1Q9_55qCmuJxx4Hi5cXR9yExXFPVQCYL1c,sdgTQOOEEBvxsKzEYUE4Vll5Pf8wC6So-27l9qDSrV5Gaf...,NA1_5226766682,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
4,ykcp8Dwwqm33s1Q9_55qCmuJxx4Hi5cXR9yExXFPVQCYL1c,sdgTQOOEEBvxsKzEYUE4Vll5Pf8wC6So-27l9qDSrV5Gaf...,NA1_5226500664,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
5,JQjDqBq2YQsVkGNQ6--rmQrgQsTdCmqsqO5ooHLeEiqa-G...,jpqj_KsazPsSOhzGcCxZdF0rRuCWcVUFCqsB_A9p1FU5Wl...,NA1_5224738854,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
6,JQjDqBq2YQsVkGNQ6--rmQrgQsTdCmqsqO5ooHLeEiqa-G...,jpqj_KsazPsSOhzGcCxZdF0rRuCWcVUFCqsB_A9p1FU5Wl...,NA1_5224663404,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
7,JQjDqBq2YQsVkGNQ6--rmQrgQsTdCmqsqO5ooHLeEiqa-G...,jpqj_KsazPsSOhzGcCxZdF0rRuCWcVUFCqsB_A9p1FU5Wl...,NA1_5224632058,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
8,JQjDqBq2YQsVkGNQ6--rmQrgQsTdCmqsqO5ooHLeEiqa-G...,jpqj_KsazPsSOhzGcCxZdF0rRuCWcVUFCqsB_A9p1FU5Wl...,NA1_5223808666,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."
9,JQjDqBq2YQsVkGNQ6--rmQrgQsTdCmqsqO5ooHLeEiqa-G...,jpqj_KsazPsSOhzGcCxZdF0rRuCWcVUFCqsB_A9p1FU5Wl...,NA1_5223789426,"{""metadata"": {""dataVersion"": ""2"", ""matchId"": ""..."


Next step:
Aggregate all data at the champion level
Each champion is a row, each column is a stat, start with total kills, total games, kills per game
    First need to identify lvl and indentation in the json output
